In [1]:
# html 코드 그대로 크롤링 할시 주석도 같이 추출됨

from bs4 import BeautifulSoup
import sys
import requests
import re
import pandas as pd
import csv
import os
import openai

f_name = "경영학부_data"
number_html = 5
url = """http://college.kau.ac.kr/web/pages/gc41789h.do
http://college.kau.ac.kr/web/pages/gc20531h.do
http://college.kau.ac.kr/web/pages/gc78574h.do
http://college.kau.ac.kr/web/pages/gc31737h.do
http://college.kau.ac.kr/web/pages/gc399h.do
http://college.kau.ac.kr/web/pages/gc19101h.do
http://college.kau.ac.kr/web/pages/gc86536h.do
http://college.kau.ac.kr/web/pages/gc48026h.do
http://college.kau.ac.kr/web/pages/gc62399h.do
"""

openai.api_key ="API_KEY"

def split_sentences(text):
    sentences = text.split('\n')  # newline('\n')을 기준으로 문자열을 나눕니다.
    sentences = [sentence.strip() for sentence in sentences]  # 각 문장 앞뒤에 있는 공백을 제거합니다.
    sentences = [sentence for sentence in sentences if sentence]  # 빈 문장을 제거합니다.
    return sentences

def extract_tables(html_str):
    # 정규표현식 패턴 정의
    pattern = r'(<table.*?>.*?</table>)'
    # 정규표현식으로 <table>...</table> 부분 찾기
    tables = re.findall(pattern, html_str, re.DOTALL)  # re.DOTALL은 줄바꿈 문자도 포함해서 찾도록 합니다.
    return tables

def table_function(table_g):
    sys = f"""다음의 맥락들을 사용하여 마지막 답변을 생성하세요. 답변을 꾸며내려 하지 마십시오.
답변을 제공할 때는 다음 형식으로 제공되어야 합니다:
테이블: [테이블 태그와 내용이 주어짐]
답: [테이블 내용 반환]

답을 결정하는 방법:

- 테이블에 있는 내용으로만 답변을 생성합니다.
- 충분한 수준의 자세한 내용을 포함하여 답변을 생성합니다.
- 자신감이 과하지 않도록 주의하세요!

예시 #1
테이블:
---------
<table class="table_nrm center">
    <tbody><tr>
        <th>이수구분</th>
        <th>학점인정</th>
        <th>비고</th>
    </tr>
    <tr>
        <td>교양 선택</td>
        <td>1학점</td>
        <td>“P”(Pass), “NP”(Non-Pass)</td>
    </tr>
</tbody></table>
---------
답 : 
---------
이수구분 : 교양선택
학점인정 : 1학점
비고 :“P”(Pass), “NP”(Non-Pass)
---------

예시 #2
테이블:
---------
<table class="table_nrm center">
    <tbody><tr>
        <th>대영역</th>
        <th>세부영역</th>
        <th colspan="2">학점</th>
    </tr>
    <tr>
        <td>필수교양</td>
        <td>
            국어<br>
            영어<br>
            소프트웨어<br>
            항공우주특성화
        </td>
        <td colspan="2">
            공과대학 13 학점<br>
            항경대학 12 학점
        </td>
    </tr>
    <tr>
        <td>전공기초교양</td>
        <td>전공기초</td>
        <td colspan="2">0 ~ 18학점, 각 학부(과)별 상이</td>
    </tr>
    <tr>
        <td>핵심교양</td>
        <td>
            1) 철학과 역사<br>
            2) 문학과 예술<br>
            3) 인간과 사회<br>
            4) 자연과 과학<br>
            5) 실용 및 글로벌
        </td>
        <td>9 학점 이상<br>학부(과)별 지정영역이수<br>(4-1 참조)</td>
        <td rowspan="2">12 학점<br>※ 1) ~ 5) 학부(과)별 지정 영역 중<br>3개 선택 후 중복되지 않는 영역에서 3학점 이상 이수<br>※ 세부영역 중복, 대영역 미중복시 인정됨</td>
    </tr>
    <tr>
        <td>심화교양</td>
        <td>
            6) 철학과 역사<br>
            7) 문학과 예술<br>
            8) 인간과 사회<br>
            9) 자연과 과학<br>
            10) 실용 및 글로벌<br>
            11) 봉사와 리더십<br>
            12) 자기계발<br>
            13) 외국어(영어)
        </td>
        <td>자유이수</td>
    </tr>
    <tr>
        <th colspan="2">계</th>
        <th colspan="2">24 ~ 43 학점</th>
    </tr>
</tbody></table>
---------
답 : 
---------
대영역 : 필수교양
세부영역 : 국어, 영어, 소프트웨어, 항공우주특성화
학점 : 공과대학 13 학점, 항경대학 12 학점

대영역 : 전공기초교양
세부영역 : 전공기초
학점 : 0 ~ 18학점, 각 학부(과)별 상이

대영역 : 핵심교양
세부영역 : 
1) 철학과 역사
2) 문학과 예술
3) 인간과 사회
4) 자연과 과학
5) 실용 및 글로벌
학점 : 9 학점 이상, 학부(과)별 지정영역이수 (4-1 참조)
비고 : 12 학점(1) ~ 5) 학부(과)별 지정 영역 중 3개 선택 후 중복되지 않는 영역에서 3학점 이상 이수, 세부영역 중복, 대영역 미중복시 인정됨

대영역 : 심화교양
세부영역 : 
6) 철학과 역사
7) 문학과 예술
8) 인간과 사회
9) 자연과 과학
10) 실용 및 글로벌
11) 봉사와 리더십
12) 자기계발
13) 외국어(영어)
학점 : 자유이수
비고 : 12 학점(1) ~ 5) 학부(과)별 지정 영역 중 3개 선택 후 중복되지 않는 영역에서 3학점 이상 이수, 세부영역 중복, 대영역 미중복시 인정됨

계 : 24 ~ 43 학점
---------

예시 #3
테이블:
---------
<table class="table_nrm">
    <tbody><tr>
        <th colspan="2">구분</th>
        <th>비고</th>
    </tr>
    <tr>
        <td class="ta_center">공통서류</td>
        <td class="ta_center">전과신청서 각 1부<br>성적증명서 1부</td>
        <td class="ta_center" rowspan="3">전과 희망 학부(과) 사무실</td>
    </tr>
    <tr>
        <td class="ta_center">경영학부<br>항공교통물류학부</td>
        <td class="ta_center">토익성적증명서 1부</td>
    </tr>
    <tr>
        <td class="ta_center">항공운항학과</td>
        <td class="ta_center">전과지원서 1부<br>토익성적증명서 1부<br>항공신체검사증명서 1부</td>
    </tr>
</tbody></table>
---------
답 : 
---------
구분 : 공통서류 - 전과신청서 각 1부, 성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 경영학부, 항공교통물류학부 - 토익성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 항공운항학과 - 전과지원서 1부, 토익성적증명서 1부, 항공신체검사증명서 1부
비고 : 전과 희망 학부(과) 사무실
---------

시작!
테이블:
---------
{table_g}
---------
답 : 
"""
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo-16k-0613",
      messages=[
      {"role": "system", "content": f'"{sys}"'},
    ], temperature =  0.001)
    answer = completion['choices'][0]['message']['content']
    if answer.endswith('"'):
        answer = answer[:-1]
        answer = answer
    return answer 

# 홈페이지 주소 가져오기
# url = "https://kau.ac.kr/web/pages/gc15707h.do"
url_split = split_sentences(url)
url_list = []

for i in range(len(url_split)):
    url_list.append(url_split[i].strip())
print(url_list)

for jj in range(len(url_list)):
#     if jj == 6:
#         f = open(f'{f_name}.csv','a', newline='' , encoding="utf-8-sig")
#         answer_db = csv.writer(f)
#         answer_db.writerow([str(jj), "", url_list[jj]])
#         f.close()
#         continue
    temp_url = url_list[jj]
    html = requests.get(temp_url)
    bs_html = BeautifulSoup(html.content, "html.parser")

    # print(bs_html) # 웹 페이지 출력
    bs_html = str(bs_html)
    split_text = split_sentences(bs_html)
    tp = 0
    tp_noise = 0
    tp_annotation = 0
    temp_list = []
    for i in range(len(split_text)):
        temp_text = split_text[i]
        if "<html lang" in temp_text and tp < number_html:
            tp += 1
        elif "<html lang" in temp_text and tp == number_html:
            temp_list.append(temp_text)
#             print(temp_text)
        elif tp == number_html and "</html>" in temp_text:
            break
        elif tp == number_html and "<!--" in temp_text and "-->" in temp_text:
            temp_text = re.sub(r'<!--.*?-->', '', temp_text)
            temp_list.append(temp_text)            
        elif tp == number_html and "<!--" in temp_text and "-->" not in temp_text:
            tp_annotation = 1
            continue
        elif tp == number_html and tp_annotation == 1 and "-->" not in temp_text:
            continue
        elif tp == number_html and tp_annotation == 1 and "-->" in temp_text:
            tp_annotation = 0
            continue
        elif tp == number_html:
            temp_list.append(temp_text)
#             print(temp_text)

    print("%%%%%%%%%%%%%%%%%%%")
    
    # table 태그는 하나의 리스트 원소로 두기
    
    table_tp = 0
    table_temp_str = ""
    re_table_list = []
    for i in range(len(temp_list)):
        if "<table" in temp_list[i] and  "</table>" in temp_list[i] and table_tp == 0:  # 한 줄 table 일 때
            test_str = temp_list[i]
            tables = extract_tables(test_str)
            for table in tables:
                # 해당 table이 test_str에서 시작하는 위치 찾기
                start_index = test_str.find(table)
                # 해당 table의 내용을 출력하기 전까지의 test_str의 내용을 출력
                re_table_list.append(test_str[:start_index].strip())
                # 해당 table 출력
                re_table_list.append(table.strip())
                # 이미 출력된 부분을 test_str에서 제거
                test_str = test_str[start_index + len(table):]
                # 남은 test_str에서 내용 출력
            re_table_list.append(test_str.strip())
        elif "<table" in temp_list[i] and table_tp == 0:
            table_tp = 1
            table_temp_str = table_temp_str + temp_list[i] + "\n"            
        elif "</table>" not in temp_list[i] and table_tp == 1:
            table_temp_str = table_temp_str + temp_list[i] + "\n"
        elif "</table>" in temp_list[i] and table_tp == 1:
            table_tp = 0
            table_temp_str = table_temp_str + temp_list[i] + "\n"
            re_table_list.append(table_temp_str.strip())
            table_temp_str = ""
        else:
            re_table_list.append(temp_list[i])
    
    print(f"-------{jj}-------")
    print(len(re_table_list))
    print(re_table_list)
# #     # table 태그 임의로 변형 (미완성)
    
#     re_table_list2 = []
#     table_temp_str = ""
#     tr_tp = 0
#     for i in range(len(re_table_list)):
#         if "<table" in re_table_list[i]:
#             table_tag_context = split_sentences(re_table_list[i])
#             for j in range(len(table_tag_context):
#                 if "<tr>" not in table_tag_context[i]
#                     temp_line = re.sub('(<([^>]+)>)', '', table_tag_context[i])
#                     temp_line = temp_line.strip()    
#                     table_temp_str = table_temp_str + temp_line + "\n"
#                 elif "<tr>" in table_tag_context[i]
#                     tr_tp = 1
#                     temp_line = re.sub('(<([^>]+)>)', '', table_tag_context[i])
#                     temp_line = temp_line.strip()    
#                     table_temp_str = table_temp_str + temp_line
#         else:
#             re_table_list2.append(re_table_list[i])

#  table 태그는 gpt에게 정리
    re_table_list2 = []               
    for i in range(len(re_table_list)):
        if "<table" in re_table_list[i]:
            table_g = re_table_list[i]
            err_count = 0
            try :
                answer = table_function(table_g)
            except Exception as e:
                err_count += 1
                print(err_count)
                if err_count >= 5:
                    print("예외가 연속으로 5번 발생했습니다. 프로그램을 종료합니다.")
                else:
                    print("예외가 발생했습니다. (%s)" % e)
                    answer = table_function(table_g)        
            re_table_list2.append(answer.strip())
        else:
            re_table_list2.append(re_table_list[i])
            
    previous_line = "#$%"
    temp_str = ""
    for i in range(len(re_table_list2)):
        temp_line = re.sub('(<([^>]+)>)', '', re_table_list2[i])
        temp_line = temp_line.strip()
        if previous_line == "" and temp_line == "":
            continue
        else:
            print(temp_line)
            temp_str = temp_str + temp_line + "\n"
        previous_line = temp_line
    temp_str = temp_str.strip()
    f = open(f'{f_name}.csv','a', newline='' , encoding="utf-8-sig")
    answer_db = csv.writer(f)
    answer_db.writerow([str(jj), temp_str, url_list[jj]])
    f.close()

['http://college.kau.ac.kr/web/pages/gc41789h.do', 'http://college.kau.ac.kr/web/pages/gc20531h.do', 'http://college.kau.ac.kr/web/pages/gc78574h.do', 'http://college.kau.ac.kr/web/pages/gc31737h.do', 'http://college.kau.ac.kr/web/pages/gc399h.do', 'http://college.kau.ac.kr/web/pages/gc19101h.do', 'http://college.kau.ac.kr/web/pages/gc86536h.do', 'http://college.kau.ac.kr/web/pages/gc48026h.do', 'http://college.kau.ac.kr/web/pages/gc62399h.do']
%%%%%%%%%%%%%%%%%%%
-------0-------
62
['<head>', '</head>', '<script type="text/javascript">', '$(document).ready(function() {', '});', '</script>', '<body>', '<div class="sub_conts">', '<div class="sub_tit">', '<h3>학부/과 소개</h3>', '</div>', '<div class="article_101">', '<span class="img_fl">', '<img src="/cmm_typeA/images/sample/sub_101_01.jpg"/>', '</span>', '<h3>경영학부</h3>', '<br/>', '<p><br/>경영학부는 1973년 항공관리학과로 창설되어, 1980년에 항공경영학과로 변경되었다. 1996년부터 2014년까지 경영학과로 운영되었으며, 2015년부터 경영학부로 개편되어 현재에 이르고 있다. 경영학부는 급변하는 21세기의 산업과 사회에 이바지할 수 있는 전문경영인의 양성

%%%%%%%%%%%%%%%%%%%
-------3-------
441
['<head>', '</head>', '<script type="text/javascript">', '$(document).ready(function() {', '});', '</script>', '<body>', '<div class="sub_conts">', '<div class="sub_tit">', '<h3>경영학부 운영내규</h3>', '</div>', '<div class="terms">', '<div class="chpt">', '<h4>제 1 장 총 칙</h4>', '<div class="sctn">', '<div class="artc">', '<h5>제1조 (목적)</h5>', "<p>이 운영내부규정은 경영학부(School of Business)(이하 '본 학부'로 칭함)의 교육 목표를 달성하기 위한 합리적인 학부 운영을 위해 필요한 사항을 규정함을 목적으로 한다.</p>", '</div>', '</div>', '</div>', '<div class="chpt">', '<h4>제 2 장 전공의 설치 및 운영</h4>', '<div class="sctn">', '<div class="artc">', '<h5>제2조 (전공)</h5>', '<p>본 학부에는 다음의 전공을 운영한다.</p>', '<ul>', '<li>1. 경영전공(Major in Business Administration)</li>', '<li>2. 글로벌항공경영전공(Major in Global Aviation Management)</li>', '</ul>', '</div>', '<div class="artc">', '<h5>제3조 (전공 배정)</h5>', '<ol class="prph">', '<li>', '<span class="num">①</span>', '전공의 배정은 학칙 시행세칙 제25조(학부 내 전공)에 의거하되, 1학년 과정을 이수한 후, 2학년 진급 전에 본 내규 제2조에 명시한 전공 중 하나


$(document).ready(function() {
});

경영학부 운영내규

제 1 장 총 칙

제1조 (목적)
이 운영내부규정은 경영학부(School of Business)(이하 '본 학부'로 칭함)의 교육 목표를 달성하기 위한 합리적인 학부 운영을 위해 필요한 사항을 규정함을 목적으로 한다.

제 2 장 전공의 설치 및 운영

제2조 (전공)
본 학부에는 다음의 전공을 운영한다.

1. 경영전공(Major in Business Administration)
2. 글로벌항공경영전공(Major in Global Aviation Management)

제3조 (전공 배정)

①
전공의 배정은 학칙 시행세칙 제25조(학부 내 전공)에 의거하되, 1학년 과정을 이수한 후, 2학년 진급 전에 본 내규 제2조에 명시한 전공 중 하나를 배정받아야 한다. 전공배정은 본인의 원에 의함을 우선으로 하며, 전공배정을 신청하지 않은 학생은 학부장이 임의로 배정한다. 단, 2019학년도 1학기 전공배정부터는 경영전공(단일전공)만 배정 한다. &lt;개정 2019.03.01.&gt;

②
전입생(복학생, 전과생 및 편입생, 재입학생)의 본 학부 전공배정 신청은 아래 각 호를 따른다.1. 복학생 ; 복학생은 2학년 1학기부터 복학신청과 함께 전공배정을 신청하여야 한다.2. 편입생 : 3학년 편입생의 경우, 편입생 수강신청 전까지 전공배정을 신청하여야 한다.3. 전과생 : 전과생은 전과 승인 이후, 수강신청 전까지 전공배정을 신청하여야 한다.4. 재입학생 : 재입학생은 2학년 1학기부터 재입학 허가 후, 전공배정 기간에 전공배정을   신청하여야 한다.

제4조 (전공의 변경)

①
전공의 변경은 학칙 시행세칙 제25조(학부 내 전공)에 따른다. 단, 전공을 변경한 학생은 변경된 전공에서 지정하는 전공과목 이수 요구조건을 만족하여야 한다. 전공이 변경된 학생의 기 이수한 학점 중 변경된 전공에서 지정한 전공과목을 이미 이수한 경우에는 전공 지정과목으로 인정할 수 있다.

②
전공변


$(document).ready(function() {
});

교과과정

교과목 체계도
교과편성 및 학점
복수전공 / 부전공

교과목 체계도

경영학부

Code Share 과목

교과편성 및 학점

경영전공

이수구분 : 교필
교과목명 : 꿈꾸는 인공지능
시간(학점) : 1(1)
1학년 1학기 : 겨울
1학년 2학기 : 여름
비고 : 2022학번 이후 수강, 온라인 강좌

이수구분 : 교필
교과목명 : 세상을 바꾸는 SW
시간(학점) : 1(1)
1학년 1학기 : 겨울
1학년 2학기 : 여름
비고 : 2022학번 이후 수강, 온라인 강좌

이수구분 : 교필
교과목명 : 인공지능소개
시간(학점) : 2(2)
2학년 1학기 : ○
비고 : 2021학번 이후 수강

이수구분 : 교필
교과목명 : 경영학입문
시간(학점) : 3(3)
1학년 1학기 : ○
비고 : 2020학번 이후 수강

이수구분 : 교필
교과목명 : 경영학도를 위한 대학수학
시간(학점) : 3(3)
1학년 1학기 : ○
비고 : 2020학번 이후 수강

이수구분 : 교필
교과목명 : KAU SPACE 인성과 진로
시간(학점) : 1(1)
1학년 1학기 : ○
비고 : 2020학번 이후 수강

이수구분 : 교필
교과목명 : 영어 커뮤니케이션Ⅰ
시간(학점) : 2(2)
1학년 1학기 : ○
비고 : 2020학번 이후 수강

이수구분 : 교필
교과목명 : 컴퓨팅적사고와 문제해결
시간(학점) : 3(2)
1학년 1학기 : ○
1학년 2학기 : ○
비고 : 2020학번 이후 수강

이수구분 : 교필
교과목명 : 항공우주산업개론
시간(학점) : 2(2)
1학년 1학기 : ○
1학년 2학기 : ○
비고 : 2019학번 이전, 2023학번 이후 수강

이수구분 : 교필
교과목명 : 항공우주학개론
시간(학점) : 2(2)
1학년 1학기 : ○
1학년 2학기 : ○
비고 : 2019학번 이전, 2023학번 이후 수강

이수구분 : 교필
교과목명 : 통계학입문
시간(학점) : 3(3)
2학년 1학기 : ○
비고 :


$(document).ready(function() {
});

졸업요건

신입학

공통 졸업요건
• 총 이수학점 : 130학점 이상(2012학번까지 : 140학점 이상)
• 전 학년 성적 평점평균 2.00 이상
※단, 조기졸업은 4.00 이상 / 학석사연계과정생은 3.50이상
• 졸업인증 통과

2017학년도 이전 신입생 : 영어인증
2018학년도 이후 신입생 : SPACE 인증 중 2개 이상 인증 취득(“S(글로벌소통역량인증)”분야는 필수 인증 취득)※ 2023학년도 이후 신(편)입학생부터는 졸업인증을 위한 공인외국어성적 기준이 TOEIC 800점 이상이니, 주의 바랍니다.
졸업인증 바로가기

학번별 전공 및 교양 이수요건
◎ 2016학번 이전
교과분류 : 교양영역
최소이수학점 : 교양필수 및 영역별 교양이수 (19학점)
비고 : 2013학년도 신입학생부터 적용(전문교양학점의 필수 취득요건도 만족하여야 함), 필수교양 : 10학점, 핵심교양 : 4개 영역(철학과 역사, 문학과 예술, 인간과 사회, 자연과 과학) 중 2개 영역(6학점), 일반교양 : 3학점(핵심교양 이수영역과 중복 불가)

교과분류 : 기초필수
최소이수학점 : 2학점
비고 : 정보기술개론(2018학년도 교양과목 교과과정개편에 따라 해당과목의 본수강 및 재수강은 ‘컴퓨팅적사고와 문제해결(교양필수)’ 과목으로 대체)

교과분류 : 전공필수
최소이수학점 : 재학 중 해당 학년/학기에 개설된 전공필수과목의 전 학점 취득(33학점)
비고 : 경제학원론, 경영학원론, 회계원리, 조직행동론, 마케팅관리, 재무관리, 경영과학, 인적자원관리, 경영정보관리, 국제경영학, 생산시스템관리

교과분류 : 전공(전공필수 및 전공선택)
최소이수학점 : 57학점
비고 : 2016학번 이전까지 적용

◎ 2017학번
교과분류 : 교양영역
최소이수학점 : 교양필수 및 영역별 교양이수 (19학점)
비고 : 2013학년도 신입학생부터 적용(전문교양학점의 필수 취득요건도 만족하여야 함), 필수교양 : 10학점, 핵심교양 : 4

%%%%%%%%%%%%%%%%%%%
-------8-------
28
['<head>', '</head>', '<script type="text/javascript">', '$(document).ready(function() {', '});', '</script>', '<body>', '<div class="sub_conts">', '<div class="sub_tit">', '<h3>전공소개</h3>', '</div>', '<div class="article_401">', '<span class="img">', '<img src="/cmm_typeA/images/sample/sub_401_01.jpg"/>', '</span>', '<h4>경영학부 소개</h4>', '<p>', '경영학부는 경영전공(단일전공)으로 운영됩니다. 경영전공은 경영학원론, 경제학원론, 회계원리의 기초 과목을 학습한 뒤 조직행동론, 경영과학, 인적자원관리, 마케팅관리, 재무관리, 생산시스템관리, 경영정보관리, 국제경제학 등 전공핵심과목을 이수합니다.', '<br/>', '또한, 학생들은 자신의 관심에 따라 회계학, 인사조직, 마케팅, 재무, 국제경영 및 전략, 경영정보, 계량경영 및 생산과 수리의 영역에서 다양한 전공 선택 심화 과목을 학습하거나, IT-Biz 융합전공, 항공경영융합전공, 조종융합전공 등의 다전공을 선택 이수할 수 있습니다.', '</p>', '</div>', '<div class="article_303">', '<h4>※ IT-Biz 융합전공, 항공경영융합전공, 조종융합전공,\xa0항공서비스융합전공\xa0안내 : <a href="http://samc.kau.ac.kr/index/main.php" target="_blank"><u><span style="color: blue;">항공경영융합학부 홈페이지</span></u></a></h4>', '<h4>※ AI융합경영전공안내 : <a href="http://college.kau.ac.kr/web/index.do?siteFl

In [ ]:
import re
import csv
import pandas as pd


def split_sentences(text):
    sentences = text.split('\n')  # newline('\n')을 기준으로 문자열을 나눕니다.
#     sentences = [sentence.strip() for sentence in sentences]  # 각 문장 앞뒤에 있는 공백을 제거합니다.
#     sentences = [sentence for sentence in sentences if sentence]  # 빈 문장을 제거합니다.
    return sentences


page_indexList = []
pageList = []
data = pd.read_csv('test3.csv', header=None, encoding="UTF-8")
for i in range(len(data)):
    page_indexList.append(data.iloc[i][0])
    pageList.append(data.iloc[i][1])

temp_str = ""
for i in range(len(page_indexList)):
    if page_indexList[i] == 41:
        temp_page = split_sentences(pageList[i])
        for j in range(len(temp_page)):
            if "// 팝업 띄우기" in temp_page[j]:
                temp_str = temp_str.strip()
                break
            else:
                temp_str = temp_str + temp_page[j] + "\n"
        temp_str = temp_str.strip()
        f = open(f'test3_filter.csv','a', newline='' , encoding="UTF-8")
        answer_db = csv.writer(f)
        answer_db.writerow([page_indexList[i], temp_str])
        f.close()    
        temp_str = ""
    else:
        f = open(f'test3_filter.csv','a', newline='' , encoding="UTF-8")
        answer_db = csv.writer(f)
        answer_db.writerow([page_indexList[i], pageList[i]])
        f.close()    
        
    

In [ ]:
import os
import openai

openai.api_key ="API_KEY"
table_g = """                            <table class="table_01"><tbody><tr><th>대영역</th><th>세부영역</th><th colspan="2">학점</th></tr><tr><td>공통교양</td><td>-</td><td colspan="2">10~13학점</td></tr><tr><td>학문기초교양</td><td>-</td><td colspan="2">13~21학점</td></tr><tr><td rowspan="2">선택교양</td><td>1) 철학과 역사<br>2) 문학과 예술<br>3) 인간과 사회<br>4) 자연과 과학<br>5)세계와 문화</td><td>9 학점 이상<br>(최소 3개 영역 선택)</td><td rowspan="2">12 학점<br>※ 1)~5) 영역 중 3개 선택 후 중복되지 않는 나머지 4개 영역에서 3학점 이상 이수</td></tr><tr><td>6) 소양<br>7) 자기계발</td><td>자유이수</td></tr><tr><td colspan="2">계</td><td colspan="2">39 학점 이상</td></tr></tbody></table></div></h5><h5>1-1) 2023학번 학부(과) 별 교양이수학점</h5><h5><div class="scroller"><table class="table_01"><tbody><tr><th>학부/과</th><th>공통교양</th><th>학문기초교양</th><th>선택교양</th><th>추가이수<br>교양학점*</th><th>계</th></tr><tr><td>항공우주 및 기계공학부</td><td>11</td><td>21</td><td>12</td><td>-</td><td>44</td></tr><tr><td>항공전자정보공학부</td><td>11</td><td>20</td><td>12</td><td>-</td><td>43</td></tr><tr><td>신소재공학과</td><td>11</td><td>17</td><td>12</td><td>-</td><td>40</td></tr><tr><td>소프트웨어학과</td><td>13</td><td>13</td><td>12</td><td>1</td><td>39</td></tr><tr><td>스마트드론공학과</td><td>10</td><td>19</td><td>12</td><td>-</td><td>41</td></tr><tr><td>AI자율주행시스템공학과</td><td>12</td><td>16</td><td>12</td><td>-</td><td>40</td></tr><tr><td>항공교통물류학부</td><td>11</td><td>15</td><td>12</td><td>1</td><td>39</td></tr><tr><td>항공운항학과</td><td>11</td><td>15</td><td>12</td><td>1</td><td>39</td></tr><tr><td>경영학부</td><td>11</td><td>15</td><td>12</td><td>1</td><td>39</td></tr></tbody></table></div></h5>* 졸업에 필요한 추가이수 교양학점은 공통교양, 학문기초교양, 선택교양 모든 교과목 포함됨<strong>&nbsp;&nbsp;&nbsp;<br><span style="color: rgb(0, 0, 0);"><span style="color: rgb(58, 50, 195);">※ 년도별 교양교과과정표를 확인하여 소속 학부(과)별 필수이수 과목을 확인할 것</span>(</span></strong><a href="http://college.kau.ac.kr/web/index.do?siteFlag=ct_www"><span style="color: rgb(0, 0, 0);">한국항공대학교 인문자연학부 (kau.ac.kr)</span></a><span style="color: rgb(0, 0, 0);">)</span><strong><br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </strong><h5><br>2) 2022학번<br><br> <div class="scroller"><table class="table_01"><tbody><tr><th>대영역</th><th>세부영역</th><th colspan="2">학점</th></tr><tr><td>공통교양</td><td>- </td><td colspan="2">11~13학점</td></tr><tr><td>학문기초교양</td><td>- </td><td colspan="2">12~20학점</td></tr><tr><td rowspan="2">선택교양</td><td>1) 철학과 역사<br>2) 문학과 예술<br>3) 인간과 사회<br>4) 자연과 과학<br>5)세계와 문화</td><td>9 학점 이상<br>(최소 3개 영역 선택)</td><td rowspan="2">12 학점<br>※ 1)~5) 영역 중 3개 선택 후 중복되지 않는 나머지 4개 영역에서 3학점 이상 이수</td></tr><tr><td>6) 소양     <br>7) 자기계발</td><td>자유이수</td></tr><tr><td colspan="2">계</td><td colspan="2">39 학점 이상</td></tr></tbody></table></div><!--2-1) 학부 ( 과 ) 별 교양이수학점 -->     </h5><h5>2-1) 2022학번 학부(과) 별 교양이수학점</h5><h5><div class="scroller"><table class="table_01"><tbody><tr><th>학부/과</th><th>공통교양</th><th>학문기초교양</th><th>선택교양</th><th>추가이수<br>교양학점*</th><th>계</th></tr><tr><td>항공우주 및 기계공학부</td><td>11</td><td> 20 </td><td> 12 </td><td>- </td><td> 43</td></tr><tr><td>항공전자정보공학부</td><td>11</td><td> 18</td><td> 12 </td><td> - </td><td>41</td></tr><tr><td>신소재공학과</td><td>11</td><td> 15</td><td> 12 </td><td>1</td><td> 39 </td></tr><tr><td>소프트웨어학과</td><td>13</td><td> 12</td><td> 12 </td><td>2</td><td> 39 </td></tr><tr><td>스마트드론공학과</td><td>12</td><td> 18</td><td> 12 </td><td>-</td><td>42</td></tr><tr><td>AI자율주행시스템공학과</td><td>12</td><td>15</td><td> 12 </td><td>-</td><td> 39 </td></tr><tr><td>항공교통물류학부</td><td>11</td><td> 13 </td><td> 12 </td><td>3</td><td> 39 </td></tr><tr><td>항공운항학과</td><td>11</td><td> 13 </td><td> 12 </td><td>3</td><td> 39 </td></tr><tr><td>경영학부</td><td>11</td><td> 13 </td><td> 12 </td><td>3</td><td> 39 </td></tr></tbody></table><p>* 졸업에 필요한 추가이수 교양학점은 공통교양, 학문기초교양, 선택교양 모든 교과목 포함됨<br><br><br>3) 2021학번<br><br> </p><div class="scroller"><table class="table_01"><tbody><tr><th>대영역</th><th>세부영역</th><th colspan="2">학점</th></tr><tr><td>공통교양</td><td>- </td><td colspan="2">9~11학점</td></tr><tr><td>학문기초교양</td><td>- </td><td colspan="2">12~20학점</td></tr><tr><td rowspan="2">선택교양</td><td>1) 철학과 역사<br>2) 문학과 예술<br>3) 인간과 사회<br>4) 자연과 과학<br>5)세계와 문화</td><td>9 학점 이상<br>(최소 3개 영역 선택)</td><td rowspan="2">12 학점<br>※ 1)~5) 영역 중 3개 선택 후 중복되지 않는 나머지 4개 영역에서 3학점 이상 이수</td></tr><tr><td>6) 소양     <br>7) 자기계발</td><td>자유이수</td></tr><tr><td colspan="2">계</td><td colspan="2">39 학점 이상</td></tr></tbody></table></div><!--2-1) 학부 ( 과 ) 별 교양이수학점 -->     </div></h5><h5>3-1) 2021학번 학부(과) 별 교양이수학점</h5><h5><div class="scroller"><table class="table_01"><tbody><tr><th>학부/과</th><th>공통교양</th><th>학문기초교양</th><th>선택교양</th><th>추가이수<br>교양학점*</th><th>계</th></tr><tr><td>항공우주 및 기계공학부</td><td>9 </td><td> 20 </td><td> 12 </td><td>- </td><td> 41</td></tr><tr><td>항공전자정보공학부</td><td>9 </td><td> 18</td><td> 12 </td><td>-</td><td> 39 </td></tr><tr><td>신소재공학과</td><td>9</td><td> 15</td><td> 12 </td><td>3</td><td> 39 </td></tr><tr><td>소프트웨어학과</td><td>11</td><td>12</td><td> 12 </td><td>4</td><td> 39 </td></tr><tr><td>스마트드론공학과</td><td>10</td><td>15</td><td> 12 </td><td>2</td><td> 39</td></tr><tr><td>항공교통물류학부</td><td>9</td><td> 13 </td><td> 12 </td><td> 5 </td><td> 39 </td></tr><tr><td>항공운항학과</td><td>9 </td><td> 13 </td><td> 12 </td><td> 5 </td><td> 39 </td></tr><tr><td>경영학부</td><td>9 </td><td> 13 </td><td> 12 </td><td> 5 </td><td> 39 </td></tr><tr></tr></tbody></table><p>* 졸업에 필요한 추가이수 교양학점은 공통교양, 학문기초교양, 선택교양 모든 교과목 포함됨<br><br>4) 2020 학번</p></div></h5><strong>
                            </strong><strong>
                                </strong><strong>
                                    </strong><strong>
                                    </strong><strong>
                                    </strong><table class="table_nrm center"><tbody><tr><th>대영역</th><th>세부영역</th><th colspan="2">학점</th>
                                </tr>
                                <tr>
                                    <td>공통교양</td>
                                    <td>-</td>
                                    <td colspan="2">7 학점</td>
                                </tr>
                                <tr>
                                    <td>학문기초교양</td>
                                    <td>-</td>
                                    <td colspan="2">12 ~ 20학점, 각 학부(과)별 상이</td>
                                </tr>
                                <tr>
                                    <td rowspan="2">선택교양</td>
                                    <td>
                                        1) 철학과 역사<br>
                                        2) 문학과 예술<br>
                                        3) 인간과 사회<br>
                                        4) 자연과 과학<br>
                                        5) 세계와 문화
                                    </td>
                                    <td>9 학점 이상<br>(최소 3개 영역 선택)</td>
                                    <td rowspan="2">12 학점<br>※ 1) ~ 5) 영역 중 3개 선택 후 중복되지 않는 영역에서 3학점 이상 이수</td>
                                </tr>
                                <tr>
                                    <td>
                                        6) 소양<br>
                                        5) 자기계발
                                    </td>
                                    <td>자유이수</td>
                                </tr>
                                <tr>
                                    <th colspan="2">계</th>
                                    <th colspan="2">39 학점 이상</th>
                                </tr>
                            </tbody></table>"""

sys = f"""다음의 맥락들을 사용하여 마지막 답변을 생성하세요. 답변을 꾸며내려 하지 마십시오.
답변을 제공할 때는 다음 형식으로 제공되어야 합니다:
테이블: [테이블 태그와 내용이 주어짐]
답: [테이블 내용 반환]

답을 결정하는 방법:

- 테이블에 있는 내용으로만 답변을 생성합니다.
- 충분한 수준의 자세한 내용을 포함하여 답변을 생성합니다.
- 자신감이 과하지 않도록 주의하세요!

예시 #1
테이블:
---------
<table class="table_nrm center">
    <tbody><tr>
        <th>이수구분</th>
        <th>학점인정</th>
        <th>비고</th>
    </tr>
    <tr>
        <td>교양 선택</td>
        <td>1학점</td>
        <td>“P”(Pass), “NP”(Non-Pass)</td>
    </tr>
</tbody></table>
---------
답 : 
---------
이수구분 : 교양선택
학점인정 : 1학점
비고 :“P”(Pass), “NP”(Non-Pass)
---------

예시 #2
테이블:
---------
<table class="table_nrm center">
<tbody>
  <tr>
    <th>편의시설</th>
    <th>위치</th>
    <th>전화번호</th>
  </tr>
  <tr>
    <td>식당</td>
    <td>1층 학생식당 내</td>
    <td>-</td>
  </tr>
  <tr>
    <td>편의점(GS)</td>
    <td>1층 학생식당 내</td>
    <td>02-3158-0336</td>
  </tr>
  <tr>
    <td>우리은행</td>
    <td>1층(운영시간) 09:30~15:30</td>
    <td>02-3158-3656</td>
  </tr>
  <tr>
    <td>블루포트(take out coffee)</td>
    <td>1층 중앙로비</td>
    <td>-</td>
  </tr>
</tbody>
</table>
---------
답 : 
---------
편의시설 : 식당
위치 : 1층 학생식당 내
전화번호 : -

편의시설 : 편의점(GS)
위치 : 1층 학생식당 내
전화번호 : 02-3158-0336

편의시설 : 우리은행
위치 : 1층(운영시간) 09:30~15:30
전화번호 : 02-3158-3656

편의시설 : 블루포트(take out coffee)
위치 : 1층 중앙로비
전화번호 : -
---------

예시 #3
테이블:
---------
<table class="table_nrm">
    <tbody><tr>
        <th colspan="2">구분</th>
        <th>비고</th>
    </tr>
    <tr>
        <td class="ta_center">공통서류</td>
        <td class="ta_center">전과신청서 각 1부<br>성적증명서 1부</td>
        <td class="ta_center" rowspan="3">전과 희망 학부(과) 사무실</td>
    </tr>
    <tr>
        <td class="ta_center">경영학부<br>항공교통물류학부</td>
        <td class="ta_center">토익성적증명서 1부</td>
    </tr>
    <tr>
        <td class="ta_center">항공운항학과</td>
        <td class="ta_center">전과지원서 1부<br>토익성적증명서 1부<br>항공신체검사증명서 1부</td>
    </tr>
</tbody></table>
---------
답 : 
---------
구분 : 공통서류 - 전과신청서 각 1부, 성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 경영학부, 항공교통물류학부 - 토익성적증명서 1부
비고 : 전과 희망 학부(과) 사무실

구분 : 항공운항학과 - 전과지원서 1부, 토익성적증명서 1부, 항공신체검사증명서 1부
비고 : 전과 희망 학부(과) 사무실
---------

시작!
테이블:
---------
{table_g}
---------
답 : 
"""
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k-0613",
  messages=[
  {"role": "system", "content": f'"{sys}"'},
])
answer = completion['choices'][0]['message']['content']
if answer.endswith('"'):
    answer = answer[:-1]
    answer = answer
print(answer)

In [ ]:
ass1 = """
함수 이름: [table_to_text]
입력: ["<table></table>"]
규칙: [당신이 정리하는 역할을 하길 바랍니다. 나는 <table> 태그를 포함한 입력 양식을 제공할 것이며 당신은 <table>을 감지하고 텍스트로 정리할것입니다.]
table_to_text(<table class="table_nrm center">
                                    <tbody><tr>
                                        <th rowspan="2">증명서 종류</th>
                                        <th rowspan="2">발급방법</th>
                                        <th colspan="2">수수료</th>
                                        <th rowspan="2">발급 시 관련부서</th>
                                    </tr>
                                    <tr>
                                        <th>국문</th>
                                        <th>영문</th>
                                    </tr>
                                    <tr>
                                        </tr><tr>
                                            <td>성적/재학/휴학/재적증명서<br>제적/졸업예정/졸업/수료증명서</td>
                                            <td>인터넷발급(<a style="color: blue;" href="http://hangkong.certpia.com/" target="_blank"><b>바로가기클릭☞</b></a>), <br>학생회관발급기<br>주민센터 내 FAX민원, <br>우체국 우편발급</td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>재학 1,200원<br>졸업 1,500원</td>
                                            <td>교무팀, 대학원행정실</td>
                                        </tr>
                                        <tr>
                                            <td>학적부, 사실증명원</td>
                                            <td>방문발급</td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>교무팀, 대학원행정실</td>
                                        </tr>
                                        <tr>
                                            <td>장학수혜확인서</td>
                                            <td>인터넷발급(<a style="color: blue;" href="http://hangkong.certpia.com/" target="_blank"><b>바로가기클릭☞</b></a>), <br>학생회관발급기<br>* 대학원은 방문 발급만 해당</td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>재학 1,200원<br>졸업 1,500원</td>
                                            <td>학생지원팀, 대학원행정실</td>
                                        </tr>
                                        <tr>
                                            <td>교육비납입증명서</td>
                                            <td>종합정보시스템에서 출력가능</td>
                                            <td>없음</td>
                                            <td>영문발급 無</td>
                                            <td>재무팀</td>
                                        </tr>
                                        <tr>
                                            <td>교외장학지원 관련 추천서</td>
                                            <td>방문발급</td>
                                            <td>없음</td>
                                            <td>영문발급 無</td>
                                            <td>학생지원팀, 대학원행정실</td>
                                        </tr>
                                        <tr>
                                            <td>실무경력확인서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>항공교통관제교육원</td>
                                        </tr>
                                        <tr>
                                            <td>비행경력증명서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>비행교육원</td>
                                        </tr>
                                        <tr>
                                            <td>비행교육원 이수증명서</td><td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>비행교육원</td>
                                        </tr>
                                        <tr>
                                            <td>공학교육인증 참여확인서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급</td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>공학교육혁신센터</td>
                                        </tr>
                                        <tr>
                                            <td>항공정비사 정비실습확인서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>항공우주 및 기계공학부</td>
                                        </tr>
                                        <tr>
                                            <td>운항관리경력증명서</td>
                                            <td>인터넷발급<br>학생회관발급기, 방문발급<br></td>
                                            <td>재학 600원<br>졸업 1,000원</td>
                                            <td>영문발급 無</td>
                                            <td>비행교육원<br></td>
                                        </tr>
                                </tbody></table>)
"""

In [ ]:
import os
import openai

openai.api_key ="API_KEY"


# 테이블 태그 임의로 지정하기
table_g = """<table class="table_nrm center">
<tbody>
  <tr>
    <th>편의시설</th>
    <th>위치</th>
    <th>전화번호</th>
  </tr>
  <tr>
    <td>식당</td>
    <td>1층 학생식당 내</td>
    <td>-</td>
  </tr>
  <tr>
    <td>편의점(GS)</td>
    <td>1층 학생식당 내</td>
    <td>02-3158-0336</td>
  </tr>
  <tr>
    <td>우리은행</td>
    <td>1층(운영시간) 09:30~15:30</td>
    <td>02-3158-3656</td>
  </tr>
  <tr>
    <td>블루포트(take out coffee)</td>
    <td>1층 중앙로비</td>
    <td>-</td>
  </tr>
</tbody>
</table>"""


# (텍스트 + 테이블 태그)를 입력으로 주었을 때 문제가 발생하는지 파악
full = f"""               

사회봉사 과목이수안내
1. 사회봉사 이수방법
이론교육(4시간) + 봉사활동(34시간 이상)

{table_g}

※ 수강신청 시 본인 최대수강신청 학점에 미포함
※ 재학 중 1회 1학점씩 총 2회 2학점까지 이수 가능
※ 성적산출시 취득학점에 포함, 평점평균 산정에 미포함
※ 연속 2회 “NP”일 경우 수강신청 불가

3. 사회봉사활동 인정기간
수강신청 직전학기 기말시험 시작일 ~ 수강신청 학기 기말시험 전까지(※ 헌혈은 기간제한 없음)

4. 사회봉사활동 기관 인정 범위
가. 시·도 및 시·군·구 자원봉사센터, 청소년활동진흥센터, 기타 사회복지관, 지역아동센터, 사회복지법인 사회복지봉사활동인증관리(VMS) 및
       1365자원봉사포털에 등록되어 ‘사회복지 봉사활동 실적 인증서(VMS)’ 및 ‘자원봉사활동 확인서(1365)’가 출력되는 기관
VMS 출력 시 필히 ‘봉사자 실적 내역정보’ 출력하여 제출
니. 비영리민간단체(봉사활동확인서 제출 시 ‘비영리민간단체등록증’ 사본 첨부)
다. 해외봉사 : 인증된 국제 NGO단체
NGO(Non-Governmental Organization 비정부기구)는 지역-국가-국제적으로 조직된 자발적인 비영리 시민단체
라. 자원봉사센터 및 시민단체와 연계하여 하는 농촌봉사활동
마. 총학생회 주관 자원봉사
1학기 : 항공산타, 신입생새로배움터, 은익제 등
2학기 : 항공제, 하계방학 중 실시 한 사회봉사활동 등
바. 기타 부서 주관 대외행사 및 교내 자원봉사
입학관리팀 ‘모교입시 설명회’, 국제교류부 ‘Global-Buddy’, 항공우주박물관 ‘안내봉사’ 등
사. 헌혈활동 : 헌혈증 제출
1회 기준 4시간 인정
1학기 최대 2회(8시간)까지만 인정
추후 본교 재학생 중 필요한 학생 및 저소득층, 백혈병 소아암 협회, 관련 사회복지기관 등에게'항공대 재학생' 이름으로 기증

5. 사회봉사활동 미인정 범위
가. 공공기관, 행정기관에서의 봉사활동
공공기관 : 주민센터(구. 동사무소), 시.도.군.구청, 경찰서, 세무서, 소방서, 우체국 등
정부행정기관 : 국세청, 법원, 감사원, 국회 등
나. 도서관(단, 시각장애인을 위한 점자도서관은 인정), 어린이집, 경로당, 종교관련 봉사활동
다. 시·군·구 개최, 지역 개최 봉사활동(단, 교육과학기술부에서 공문화 되어 온 봉사활동은 인정)
라. 제출기간까지 봉사활동확인서가 미발급되거나 기간 및 시간이 정확히 기재되지 않은 봉사활동
   (예 : 중앙일보 '공부의 신')

6. 제출서류
가. “사회봉사”과목 학점인정 서류
수강신청 전원 제출, 미제출시 학점인정 안됨
본교 총학생회 주관 자봉단 활동 시에도 제출
나. 사회봉사활동확인서(봉사활동을 실시한 기관에 양식이 없을 경우 사용)

7. 유의사항
가. 필히 ‘이론교육’ 4시간을 이수해야 봉사활동확인서 제출 시 학점 인정
나. 봉사활동 확인서는 원본을 제출하여야 하며, 제출한 서류는 일체 반환하지 않음
다. 봉사활동 확인서 수정 금지 (수정 시 담당자 날인이 없을 경우 무효 처리)
    * 문의사항 : 02-300-0014
"""


# 테이블과 관련된 질문하기
question ="""혹시 편의시설에 대해 알려줄래?"""



sys = f"""{full}

{question}"""

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo-16k-0613",
  messages=[
  {"role": "system", "content": f'"{sys}"'},
])
answer = completion['choices'][0]['message']['content']
# answer = answer.replace('\"',"")
print(answer)

In [ ]:
import pandas as pd
import openai
import csv
import numpy as np
from time import sleep
import json
import ast
import re
import random
from itertools import *
import sys
delay = 2

openai.api_key ="API_KEY"

def get_embedding(text, model="text-embedding-ada-002"):
    count = 0 # 예외 발생 횟수를 저장할 변수
    max_count = 5 # 예외가 발생한 최대 횟수
    while True:
        try :
            temp = openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']
            count=0
            return temp
        except Exception as e:
            count += 1
            print(count)
            if count >= max_count:
                print("예외가 연속으로 %d번 발생했습니다. 프로그램을 종료합니다." % max_count)
                break
            else:
                print("예외가 발생했습니다. (%s)" % e)
def split_sentences(text):
    sentences = text.split('\n')  # newline('\n')을 기준으로 문자열을 나눕니다.
    sentences = [sentence.strip() for sentence in sentences]  # 각 문장 앞뒤에 있는 공백을 제거합니다.
    sentences = [sentence for sentence in sentences if sentence]  # 빈 문장을 제거합니다.
    return sentences

                
Sec_index = []
SecList = []
    
csv_name = "test3_filter"

data = pd.read_csv(f'{csv_name}.csv', header=None, encoding="UTF-8")
for i in range(len(data)):    
    Sec_index.append(data.iloc[i][0])
    SecList.append(data.iloc[i][1])
    
###########################################################################################
temp_str = ""
for i in range(len(Sec_index)):
    print(f"------------  {i}  -------------")
    temp_manual = SecList[i]
    splitList = split_sentences(temp_manual)
    for j in range(len(splitList)):
        pattern = re.compile(r'\d+\.')
        matches = re.findall(pattern, temp_sentence)
        if matches:
            temp_str = temp_str + splitList[j]
        print(splitList[j])
        print("######################")
        